# V 0.2

This is next "version" of [simple finetuning](https://www.kaggle.com/code/yannchikk/bart-large-cnn-dialoguesum-booksum-full-finetuning). There i try train the bart-large-cnn by LoRA finetuning method with PEFT lib. 

There i going to train LoRA with processed text and text without processing and make Model Soup from this two versions. 

In this Notebook: 

- ### Train original '[bart-large-cnn](https://huggingface.co/facebook/bart-large-cnn)'

- ## without text processing + preprocessed dataset 

- ### Using peft lib for add LoRA layers to model

- ### Finetune only 'target_modules'

- ### Using DataParalell more accelerate models

In [1]:
class Config:
    
    max_length = 1024
    target_max_length = 512

    epochs = 10
    
    batch_size = 8

#     model_preset_trained = "doublecringe123/bardt-large-cnn-dialoguesum-booksum"
    
    try: 
        model_preset = model_preset_trained 
    except: 
        model_preset = "facebook/bart-large-cnn"
    
    
    lora_params = {
        'target_modules':['out_proj', 'v_proj', 'q_proj', 'cf1', 'cf2'], 
        'r':8, 
        'lora_alpha': 16, 
    }

    inp = 'input_content'
    target = 'target'

    save_limits = 1

cfg = Config()

# At First, lets load datasets



In [2]:
! pip install -q --upgrade pip
! pip install -q transformers[torch]
! pip install -q -U transformers==4.38.2 datasets==2.18.0 evaluate rouge_score

try: 
    import wandb
    wandb.init(mode='disabled')
except: 
    ...

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

This is my code from [github repo](https://github.com/goin2crazy/multy-dataset/blob/main/main.py) 

In [ ]:
! wget -O "mds.py" "https://raw.githubusercontent.com/goin2crazy/multy-dataset/main/main.py" 

# Prepare Dataset

In [4]:
from mds import NewDataset
dataset_params = {
    "knkarthick/dialogsum": ("dialogue", "summary"), 
    "doublecringe123/dialoguesum-npc-dialoguesum-stemmed-augmented": ('inp', 'target')
}

dataset = NewDataset(dataset_params, input_col_name = cfg.inp, target_col_name = cfg.target)

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split:   0%|          | 0/59070 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7000 [00:00<?, ? examples/s]

# Load model and tokenizer

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained(cfg.model_preset)
model = AutoModelForSeq2SeqLM.from_pretrained(cfg.model_preset)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

# Creating LoRA Model with PEFT

PEFT refers to a group of techniques that enable efficient adaptation of large language models (LLMs) to specific tasks or domains. It involves fine-tuning only a small subset of parameters in the LLM, rather than modifying the entire model. This approach offers several advantages:

I knew about PEFT and LoRA models buildin from [this notebook](https://www.kaggle.com/code/ajinkyabhandare2002/fine-tune-flan-t5-base-for-chat-with-peft-lora#Setup-the-PEFT/LoRA-model-for-Fine-Tuning)

In [6]:
! pip install -q peft

In [7]:
from peft import LoraConfig, get_peft_model, TaskType

lora_conf = LoraConfig(
    **cfg.lora_params, 
    lora_dropout = 0.05,
    bias = 'none', 
    task_type = TaskType.CAUSAL_LM,
    init_lora_weights = 'gaussian', 
)

In [8]:
lora_model = get_peft_model(model=model, peft_config=lora_conf)

lora_model.print_trainable_parameters()

trainable params: 1,769,472 || all params: 408,059,904 || trainable%: 0.4336304504938569


In [9]:
from torch import nn
import torch 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lora_model.model = nn.DataParallel(lora_model.model)

In [10]:
def preprocess_function(examples):
    try: 
        inputs = [doc for doc in examples[cfg.inp]]
        model_inputs = tokenizer(inputs, max_length=cfg.max_length, truncation=True)

        labels = tokenizer(text_target=examples[cfg.target], max_length=cfg.target_max_length, truncation=True)

        model_inputs["labels"] = labels["input_ids"]
        return model_inputs
    except TypeError as e:
        print(e)
        print(examples[cfg.inp])

dataset = dataset.map(preprocess_function, batched = True)
tokenized_train, tokenized_val, tokenized_test = dataset.splits

Map:   0%|          | 0/71530 [00:00<?, ? examples/s]

Map:   0%|          | 0/3500 [00:00<?, ? examples/s]

Map:   0%|          | 0/8500 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=cfg.model_preset)

2024-04-04 15:22:25.458379: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-04 15:22:25.458484: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-04 15:22:25.585341: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Define Metrics

In [12]:
import evaluate

rouge = evaluate.load("rouge")

In [13]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

# Define training arguments

In [14]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [15]:
training_args = Seq2SeqTrainingArguments(
    output_dir="bardt-large-cnn-dialoguesum-booksum-lora",
    evaluation_strategy="epoch",
#     save_strategy='steps',
#     save_safetensors = True,
#     save_steps = 100, 
    learning_rate=2e-5,
    per_device_train_batch_size=cfg.batch_size,
    per_device_eval_batch_size=cfg.batch_size,
    weight_decay=0.01,
    num_train_epochs=cfg.epochs,
    predict_with_generate=True,
    
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss


RuntimeError: 
            Some tensors share memory, this will lead to duplicate memory on disk and potential differences when loading them again: [{'model.module.final_logits_bias', 'base_model.model.final_logits_bias'}, {'base_model.model.lm_head.weight', 'model.module.model.shared.weight', 'base_model.model.model.encoder.embed_tokens.weight', 'model.module.model.decoder.embed_tokens.weight', 'model.module.model.encoder.embed_tokens.weight', 'base_model.model.model.decoder.embed_tokens.weight', 'base_model.model.model.shared.weight', 'model.module.lm_head.weight'}, {'base_model.model.model.encoder.embed_positions.weight', 'model.module.model.encoder.embed_positions.weight'}, {'model.module.model.encoder.layers.0.self_attn.k_proj.weight', 'base_model.model.model.encoder.layers.0.self_attn.k_proj.weight'}, {'base_model.model.model.encoder.layers.0.self_attn.k_proj.bias', 'model.module.model.encoder.layers.0.self_attn.k_proj.bias'}, {'model.module.model.encoder.layers.0.self_attn.v_pro

In [ ]:
model.save_pretrained("bardt-large-cnn-dialoguesum-booksum-lora")
model.push_to_hub("bardt-large-cnn-dialoguesum-booksum-lora", commit_message = f"Original+Augmented+Stemmed Dataset, {cfg.epochs} epochs")